In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.append("/Users/julianganzabal/facultad/lab-ml/mllab-tools")

In [3]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

Using TensorFlow backend.


In [424]:
model=get_deep_rnn((100, 69), dense_units=69, stateful=False)

input shape =  (100, 69)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 200)          216000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_19 (Dense)             (None, 69)                13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


## Cuanto vale T y cuanto vale D?

In [430]:
layer1 = model.get_layer('lstm_1')
layer1.get_config()['batch_input_shape']

(None, 100, 69)

In [431]:
chars = np.load('training_chars.npy')
chars_to_indices = dict((c, i) for i, c in enumerate(chars))
indices_to_chars = dict((i, c) for i, c in enumerate(chars))

In [432]:
window = 100

In [1013]:
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')

In [1014]:
def predict_phrase(model, initial_seed, window = 100, verbose=True, sample_type = 'greedy'):
    # El numeral no esta en el diccionario por lo que lo pasa a todos ceros en el one-hot
    initial_seed=initial_seed[-window:]
    initial_text = '#'*(window-len(initial_seed))+initial_seed # Padding
    X_test = chars_to_one_hot(initial_seed, chars, chars_to_indices, 100)
    probs = model.predict(X_test)
    probs_sorted_indexes = np.argsort(probs)[0][::-1]
    if verbose:
        print([(indices_to_chars[i], probs[0][i]) for i in probs_sorted_indexes[:5]])
    return probs[0]

In [1015]:
def sample_probs(probs, sample_type='greedy', temp=1):
    if (sample_type=='greedy'):
        return np.argmax(probs)
    if (sample_type=='simple'):
        return sample(probs, temp, verbose=False, return_dist=True)

## Distribución de probabilidades para:
#### "Estaba el novi"

In [573]:
probs = predict_phrase(model, "Estaba el novi")

[('o', 0.89250946), ('a', 0.082889929), ('e', 0.010628676)]


In [574]:
indices_to_chars[sample_probs(probs, sample_type='greedy')]

'o'

In [575]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[ 0.89250946  0.08288993  0.01062868]
['a', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'a', 'o', 'o', 'o', 'o']


## Distribución de probabilidades para:
#### "Estaba la novi"

In [577]:
probs = predict_phrase(model, "Estaba la novi")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('a', 0.91647363), ('o', 0.06639652), ('e', 0.0076717283)]


'a'

In [578]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[ 0.91647375  0.06639653  0.00767173]
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'o']


## Distribución de probabilidades para:
#### "Estaba la novia"

In [580]:
probs = predict_phrase(model, "Estaba la novia")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('.', 0.50201517), (' ', 0.32931387), (',', 0.039581548)]


'.'

In [581]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[ 0.50201517  0.32931387  0.03958155]
['.', ',', '.', 'r', '.', ')', '.', 'n', '.', '.', '!', ')', ' ', ' ', ' ', ' ', ' ', 'ñ', ')', ' ']


## Generamos texto
### Greedy Search

In [1135]:
def greedy_search(model, phrase, N=200):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        next_char = indices_to_chars[sample_probs(probs, sample_type='greedy')]
        phrase=phrase + next_char
        print(next_char, end='')

In [1136]:
greedy_search(model, 'NOVIA:')

NOVIA: (Seria) Ahora está con la cabeza de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caball

In [1137]:
greedy_search(model, 'LEONARDO:', 300)

LEONARDO: (Entrando) ¡Qué casa?

MADRE: (Al novio) No te puede a me puede de azahar.

MADRE: (Al novio) No te puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a la cara de la caballo de la caballo de la caballo de la caballo de la c

In [1138]:
greedy_search(model, 'JULIAN:', 300)

JULIAN: de caballo se puede de la boda.

NOVIA: (Seria) La mujer el azahar.

NOVIA: (Alegre) ¡Qué te puede a la carrel?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre

In [1139]:
# Con texto de la obra
initial_text = 'MOZO 1: (Entrando) ¡Tienes que beber con nosotros!\n\nNOVIO: Estoy esperando a la novia.\n\nMOZO 2: ¡Ya '
print(initial_text)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 


In [1140]:
greedy_search(model, initial_text, N=300)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya vinte a la cara con el caballo de azahar.)

NOVIA: (Alegre) ¡Qué te puede a la carrel?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente e

## Random Sample

In [1141]:
def simple_sample(model, phrase, temp, N=200):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        index, dist = sample_probs(probs, sample_type='simple', temp=temp)
        next_char = indices_to_chars[index]
        phrase=phrase + next_char
        print(next_char, end='')

In [1143]:
simple_sample(model, 'NOVIA:', 1, 300)

NOVIA: (La mace en caballo se los iglos pienes de la novia el agua!

NOVIA: ¿A los niño?

MADRE: ¿Tis sertido?

MADRE: (Concigue camaré la concia.)

VaCINA: Sí. Están serido más usas una irezos. Sirába tedidadore de un plator del manto.

NOVIA: Ciro una jeria.
MUJER: Vabla tu pinvión de fendore.

CRIADA: 

In [1144]:
# Cada pasada escribe algo diferente
simple_sample(model, 'NOVIA:', 1, 300)

NOVIA: ¡Señis enseguntadad!

NOVIA: (Buerte) Las fuertes parican en corona. Pero no se pude nada así.

NOVIO: ¿Son la degalla!

VECINA: (Es lo mediero)¡Tate espira muy tejoras que los vistós vedidaras. (Cogiénte la siempro. Vaevuan está que son alse.

NOVIA: Está que?

VECINA: Yo me, puedes que es una muy

In [1145]:
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) La madre de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo se la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo se la caballo de la caballo de la caballo de

In [1146]:
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) Ahora está con la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo se la caballo de la cabeza de la caballo 

In [1147]:
simple_sample(model, 'NOVIA:', 2)

NOVIA:
Mento,
cofmarúmítaspuntéjhefe suyialde
tázpbercó das vozy.
PADRE:?ALeoñeva. Fuorteié
heríobatismotena coáqioloO.!LmEntedeta,.?¿pamege! Túus mudemba.

AystO viege la réplo.
BRade:

Lossbresdr, decén.


In [1153]:
simple_sample(model, initial_text, 0.5, 300)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya no sabe con la verte!

MADRE: Sí, me hoy de hablar a la casa.

LEONARDO: ¡Me ires de cortido?

NOVIA: (Agria) ¡Qué tienes el campo con viste a la cara con el caballo de azahar!

NOVIA: (Al novio) Es que tengo es una flarte.

CRIADA: (Seria) La medre de rombre.

CRIADA: Cuando que con las primaros.



# Stochastic Beam search

https://www.youtube.com/watch?v=zPv7nWB4_ts

In [1148]:
def init_models(beam_width):
    models = []
    for i in range(beam_width):
        model = get_deep_rnn((100, 69), dense_units=69, stateful=False, verbose=False)
        model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
        models.append(model)
    return models

In [1149]:
def predict_beams(phrases, log_probs, beam_width, temp=1):
    samples = []
    log_probs_out = []
    for j, phrase in enumerate(phrases):
        for i in range(beam_width):
            probs = predict_phrase(models[i], phrase, verbose=False)
            index, dist = sample_probs(probs, sample_type='simple', temp=temp)
            samples.append(phrase + indices_to_chars[index])
            log_probs_out.append(log_probs[j] + np.log(dist[index]))
    return np.array(samples), np.array(log_probs_out)

In [1150]:
beam_width = 5
models = init_models(beam_width)

KeyboardInterrupt: 

In [1127]:
initial_phrase = 'NOVIA:'
phrases = [initial_phrase]
log_probs = np.zeros(len(phrases)*beam_width)
for i in range(100):
    print(phrases)
    phrases, log_probs = predict_beams(phrases, log_probs, beam_width, temp=2)
    #print(phrases)
    #print(log_probs)
    sorted_indexes = np.argsort(log_probs)[::-1][:beam_width]
    log_probs = log_probs[sorted_indexes]
    phrases = phrases[sorted_indexes]
    print('---------------------------------------------')
print(phrases[0])

['NOVIA:']
---------------------------------------------
['NOVIA: ' 'NOVIA: ' 'NOVIA:\n' 'NOVIA:\n' 'NOVIA:L']
---------------------------------------------
['NOVIA:\n¡' 'NOVIA:\n¡' 'NOVIA:\n¡' 'NOVIA:\nD' 'NOVIA:\nD']
---------------------------------------------
['NOVIA:\nDe' 'NOVIA:\nDe' 'NOVIA:\nDe' 'NOVIA:\nDe' 'NOVIA:\nDe']
---------------------------------------------
['NOVIA:\nDes' 'NOVIA:\nDes' 'NOVIA:\nDes' 'NOVIA:\nDes' 'NOVIA:\nDes']
---------------------------------------------
['NOVIA:\nDesp' 'NOVIA:\nDesp' 'NOVIA:\nDesp' 'NOVIA:\nDesp' 'NOVIA:\nDesp']
---------------------------------------------
['NOVIA:\nDespi' 'NOVIA:\nDespi' 'NOVIA:\nDespi' 'NOVIA:\nDespi'
 'NOVIA:\nDespi']
---------------------------------------------
['NOVIA:\nDespie' 'NOVIA:\nDespie' 'NOVIA:\nDespie' 'NOVIA:\nDespie'
 'NOVIA:\nDespie']
---------------------------------------------
['NOVIA:\nDespier' 'NOVIA:\nDespier' 'NOVIA:\nDespier' 'NOVIA:\nDespier'
 'NOVIA:\nDespier']
-------------------------

KeyboardInterrupt: 

In [1111]:
print(phrases[0])

NOVIA: (Seria) Ahora de casaria. (Se siente de la caballo de la caballo de la caballo de la casa.)

NOVIA:


## Algunas frases con Beam Search
Temp = 1

# Verificación Beam search

In [1101]:
selected_pred = 0 # Cual de las predicciones verifico. Deberia quedarme con la de mayor prob
predicted_chars = phrases[selected_pred][len(initial_phrase):]
print(predicted_chars)
log_sum = 0
predicted_phrase_parc = initial_phrase
for c in predicted_chars:
    pred_prob = predict_phrase(model, predicted_phrase_parc, verbose=False)[chars_to_indices[c]]
    print(pred_prob)
    predicted_phrase_parc = predicted_phrase_parc + c
    log_sum = log_sum + np.log(pred_prob)
log_sum

 (Ser
0.525927
0.296634
0.278172
0.86822
0.606687


-3.7784151881933212